## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

In [14]:
# 1. Import the WeatherPy_database.csv file.
weatherPy_data = "Vacation/WeatherPy_Database.csv"

In [16]:
city_data_df = pd.read_csv(weatherPy_data)
city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Puerto Ayora,EC,-0.74,-90.35,77.00,61,40,11.41,scattered clouds
1,1,Beloha,MG,-25.17,45.05,63.70,80,39,10.20,scattered clouds
2,2,Avarua,CK,-21.21,-159.78,78.80,65,53,11.41,broken clouds
3,3,Rincon De La Victoria,ES,36.72,-4.28,66.99,39,20,9.17,few clouds
4,4,Ushuaia,AR,-54.80,-68.30,46.40,61,40,20.80,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
722,722,Berbera,SO,10.44,45.01,84.49,56,0,1.70,clear sky
723,723,Portland,US,45.52,-122.68,66.20,72,20,2.39,few clouds
724,724,Zhezkazgan,KZ,47.80,67.71,49.08,65,17,5.48,few clouds
725,725,Soyo,AO,-6.13,12.37,76.26,86,96,12.95,overcast clouds


In [17]:
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [ ]:
g_key

In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Puerto Ayora,EC,-0.74,-90.35,77.00,61,40,11.41,scattered clouds
2,2,Avarua,CK,-21.21,-159.78,78.80,65,53,11.41,broken clouds
6,6,Sumenep,ID,-7.02,113.87,79.52,80,98,6.15,overcast clouds
8,8,Puri,IN,19.80,85.85,77.00,100,40,5.82,thunderstorm
10,10,Minbu,MM,20.18,94.88,77.18,90,99,4.81,overcast clouds
12,12,Hilo,US,19.73,-155.09,86.00,58,20,6.35,few clouds
14,14,Lazaro Cardenas,MX,17.96,-102.20,86.49,56,0,7.49,clear sky
17,17,Sal Rei,CV,16.18,-22.92,80.60,74,75,20.80,broken clouds
29,29,Colomi,BO,-17.35,-65.87,80.60,21,20,8.05,few clouds
34,34,Morondava,MG,-20.28,44.28,76.91,75,30,7.05,scattered clouds


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        227
City           227
Country        227
Lat            227
Lng            227
Max Temp       227
Humidity       227
Cloudiness     227
Wind Speed     227
Description    227
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Description","Lat","Lng"]].copy()
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,77.00,scattered clouds,-0.74,-90.35,
2,Avarua,CK,78.80,broken clouds,-21.21,-159.78,
6,Sumenep,ID,79.52,overcast clouds,-7.02,113.87,
8,Puri,IN,77.00,thunderstorm,19.80,85.85,
10,Minbu,MM,77.18,overcast clouds,20.18,94.88,
12,Hilo,US,86.00,few clouds,19.73,-155.09,
14,Lazaro Cardenas,MX,86.49,clear sky,17.96,-102.20,
17,Sal Rei,CV,80.60,broken clouds,16.18,-22.92,
29,Colomi,BO,80.60,few clouds,-17.35,-65.87,
34,Morondava,MG,76.91,scattered clouds,-20.28,44.28,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,77.00,scattered clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
2,Avarua,CK,78.80,broken clouds,-21.21,-159.78,Paradise Inn
6,Sumenep,ID,79.52,overcast clouds,-7.02,113.87,OYO 3271 Homestay Pandian
8,Puri,IN,77.00,thunderstorm,19.80,85.85,OYO 1577 Hotel Shree Hari Grand
10,Minbu,MM,77.18,overcast clouds,20.18,94.88,Kaung Thar Guest House - Burmese Only
...,...,...,...,...,...,...,...
706,Baiao,BR,87.01,overcast clouds,-2.79,-49.67,Hotel Eurosono
709,Nanakuli,US,84.20,few clouds,21.39,-158.15,Camp Pālehua
717,Conde,BR,80.60,few clouds,-7.26,-34.91,Pousada Beija Flor
722,Berbera,SO,84.49,clear sky,10.44,45.01,Damal Hotel


In [27]:
hotel_df.count()

City           227
Country        227
Max Temp       227
Description    227
Lat            227
Lng            227
Hotel Name     227
dtype: int64

In [30]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,77.00,scattered clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
2,Avarua,CK,78.80,broken clouds,-21.21,-159.78,Paradise Inn
6,Sumenep,ID,79.52,overcast clouds,-7.02,113.87,OYO 3271 Homestay Pandian
8,Puri,IN,77.00,thunderstorm,19.80,85.85,OYO 1577 Hotel Shree Hari Grand
10,Minbu,MM,77.18,overcast clouds,20.18,94.88,Kaung Thar Guest House - Burmese Only
...,...,...,...,...,...,...,...
706,Baiao,BR,87.01,overcast clouds,-2.79,-49.67,Hotel Eurosono
709,Nanakuli,US,84.20,few clouds,21.39,-158.15,Camp Pālehua
717,Conde,BR,80.60,few clouds,-7.26,-34.91,Pousada Beija Flor
722,Berbera,SO,84.49,clear sky,10.44,45.01,Damal Hotel


In [29]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [36]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [33]:
locations = hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
0,-0.74,-90.35
2,-21.21,-159.78
6,-7.02,113.87
8,19.80,85.85
10,20.18,94.88
...,...,...
706,-2.79,-49.67
709,21.39,-158.15
717,-7.26,-34.91
722,10.44,45.01


In [37]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))